In [ ]:
!pip install pyxlsb
!pip install pandas --upgrade

In [103]:
!pip install pandas pyxlsb numpy nltk

In [138]:
import pandas as pd
import pyxlsb
import numpy as np
from nltk.corpus import stopwords
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [124]:
dataframe1= pd.read_excel('/content/drive/MyDrive/stc TV Data Set_T3.xlsb', engine='openpyxl')

In [212]:
dataframe=dataframe1.copy()

In [213]:
dataframe.head()

,Unnamed: 0,user_id_maped,program_name,rating,date_,program_genre
0,0,26138,100 treets,1,2017-05-27,Drama
1,1,7946,Moana,1,2017-05-21,Animation
2,2,7418,The Mermaid Princess,1,2017-08-10,Animation
3,3,19307,The Mermaid Princess,2,2017-07-26,Animation
4,4,15860,Churchill,2,2017-07-07,Biography


In [214]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 6 columns):
 #   Column         Non-Null Count    Dtype         
---  ------         --------------    -----         
 0   Unnamed: 0     1048575 non-null  int64         
 1   user_id_maped  1048575 non-null  int64         
 2   program_name   1048575 non-null  object        
 3   rating         1048575 non-null  int64         
 4   date_          1048575 non-null  datetime64[ns]
 5   program_genre  1048575 non-null  object        
dtypes: datetime64[ns](1), int64(3), object(2)
memory usage: 48.0+ MB


In [215]:
dataframe['program_name']=dataframe.program_name.str.lower()

In [216]:
def remove_special_characters(input_string):
    # Define a string containing all special characters to be removed
    special_chars = "!\"#$%&'()*+,-./:;<=>?@[\\]^_`{|}~"

    # Use the translate() method to remove the special characters
    translator = str.maketrans('', '', special_chars)
    cleaned_string = input_string.translate(translator)
    return cleaned_string
dataframe['program_name'] = dataframe['program_name'].apply(remove_special_characters)

In [217]:
dataframe.head()

,Unnamed: 0,user_id_maped,program_name,rating,date_,program_genre
0,0,26138,100 treets,1,2017-05-27,Drama
1,1,7946,moana,1,2017-05-21,Animation
2,2,7418,the mermaid princess,1,2017-08-10,Animation
3,3,19307,the mermaid princess,2,2017-07-26,Animation
4,4,15860,churchill,2,2017-07-07,Biography


In [218]:
new_data = pd.DataFrame()
new_data['avg'] = dataframe.groupby('program_name')['rating'].mean()
grouped_data = dataframe.groupby('program_name')['program_genre'].unique()
new_data.reset_index(drop=False, inplace=True)
new_data = pd.merge(new_data, grouped_data, on='program_name')

In [219]:
new_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8004 entries, 0 to 8003
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   program_name   8004 non-null   object 
 1   avg            8004 non-null   float64
 2   program_genre  8004 non-null   object 
dtypes: float64(1), object(2)
memory usage: 187.7+ KB


In [220]:
new_data['program_genre'] = new_data['program_genre'].astype(str).apply(remove_special_characters)

In [232]:
new_data.head()

,program_name,avg,program_genre
0,10 days in a madhouse,1.707052,Thriller
1,100 treets,1.836305,Drama
2,101 dalmatians,1.466102,Family
3,102 dalmatians,1.558140,Comedy Action
4,116,2.018470,Drama


In [ ]:
!pip install scikit-surprise
from surprise import Reader, Dataset, KNNBasic
from surprise.model_selection import train_test_split
from surprise import accuracy

In [222]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

def get_recommendations(movie_name):
    movie_name = movie_name.lower()
    tfidf_vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1, 3))
    tfidf_matrix = tfidf_vectorizer.fit_transform(new_data['program_genre'])
    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
    idx = new_data[new_data['program_name'].str.lower() == movie_name].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    top_similar_indices = [i for i, _ in sim_scores[1:6]]
    recommended_movies = new_data.iloc[top_similar_indices].reset_index(drop=True)

    return recommended_movies


In [231]:
input_movie = "Moana"
recommendations = get_recommendations(input_movie)
recommendations.head()

,program_name,avg,program_genre
0,a tory and a proverb ep,2.148571,Animation
1,aladdin,1.978261,Animation
2,albert asks ep,2.620690,Animation
3,alvin and the chipmunks,1.783418,Animation
4,alvin and the chipmunks the queakquel,1.554562,Animation
